#### Requirements

In [4]:
import urllib
import urllib.request
from urllib.request import urlopen, Request
import urllib.parse
from urllib.error import HTTPError
from urllib.error import URLError
from http.cookiejar import CookieJar
from bs4 import BeautifulSoup
import lxml
import cchardet
import json, os
import pandas as pd
from fpdf import FPDF


#### Request

In [ ]:
base_url = "https://api.elsevier.com/content/search/scopus?"

apiKey = "a981e389772e23df3d045cc51e2bcfc6"

query = "TITLE-ABS-KEY ({multiple sclerosis} AND {corpus callosum})"

view = "STANDARD"


try:
    url = base_url + "query=" + urllib.parse.quote(query) + "&apiKey=" + apiKey + "&view=" + view
    fetch_results = json.dumps(json.loads(urlopen(url).read()), indent=4)
    with open("scopus_search_results0.json", "w") as output:
        output.write(fetch_results)
    with open("scopus_search_results0.json", "r") as search_results:
        search_results = json.load(search_results)
    total_results = int(search_results["search-results"]["opensearch:totalResults"])

    start = 26
    int(total_results/(start-1))
    
    total_files = int(total_results/(start-1))
    counter = 1
    while counter <= total_files:
        url = base_url + "query=" + urllib.parse.quote(query) + "&apiKey=" + apiKey + "&view=" + view + "&start=" + str(start)
        fetch_results = json.dumps(json.loads(urlopen(url).read()), indent=4)
        with open(f"scopus_search_results{counter}.json", "w") as output:
            output.write(fetch_results)
        if start+25 < total_results:
            start += 25
            counter += 1
        else:
            break

except HTTPError as e:
    print(e)
except URLError as e:
    print('Not found!')

#### Merge and save results

In [5]:
IDs = [None] * total_results
titles = [None] * total_results
links = [None] * total_results

counter = 0
while counter <= total_files:
    with open(f"scopus_search_results{counter}.json", "r") as search_results:
        search_results = json.load(search_results)
    for index, entry in enumerate(search_results["search-results"]["entry"]):
        IDs[index+(counter*25)] = entry["eid"]
        titles[index+(counter*25)] = entry["dc:title"]
        for link in entry["link"]:
            if link["@ref"] == "scopus":
                links[index+(counter*25)] = entry["link"][entry["link"].index(link)]["@href"]
    counter += 1

In [6]:
abstracts = [None] * total_results

for index, link in enumerate(links):
    abstracts[index] = BeautifulSoup(urllib.request.build_opener(urllib.request.HTTPCookieProcessor(CookieJar())).open(Request(link, headers = {'User-Agent': 'Mozilla/5.0'})).read(), 'lxml').select("section[id = abstractSection] > p")[0].replace_with("</p>", "")
    if index >= 5:
        break


In [9]:
articles_list = {'IDs': IDs, 'Title': titles, 'Abstract': abstracts}
articles_list = pd.DataFrame(articles_list)
articles_list.to_csv("scopus_articles.csv")